In [1]:
import pandas as pd
from datetime import date, datetime
from vnstock import *


In [2]:
# create dim_account_relation
stock_relation = pd.read_excel('stock_relation.xlsx', sheet_name='dim_account')
account_relation = pd.read_excel('stock_relation.xlsx', sheet_name='account_relation')
stock_relation

,account_key,account_name,sign,account_type,level
0,270,TỔNG TÀI SẢN,1,cdkt,1
1,100,TÀI SẢN NGẮN HẠN,1,cdkt,2
2,110,Tiền và tương đương tiền,1,cdkt,3
3,111,Tiền,1,cdkt,4
4,112,Các khoản tương đương tiền,1,cdkt,4
...,...,...,...,...,...
134,62,Lợi ích của cổ đông thiểu số,1,kqkd,1
135,61,Lợi nhuận của Cổ đông của Công ty mẹ,1,kqkd,1
136,70,Lãi cơ bản trên cổ phiếu,1,kqkd,1
137,71,Lãi trên cổ phiếu pha loãng,1,kqkd,1


In [3]:
dim_bs_relation = stock_relation[stock_relation["level"] == 4]

# Add Account column
dim_bs_relation = pd.merge(dim_bs_relation, account_relation, left_on='account_key', right_on='child', how='left')
dim_bs_relation['account_id_lv3'] = dim_bs_relation['parent']
dim_bs_relation = dim_bs_relation.drop(columns=['parent', 'child', 'level'])
merged_df = pd.merge(dim_bs_relation['account_id_lv3'], stock_relation, left_on='account_id_lv3', right_on='account_key', how='left')
dim_bs_relation["Account"] = merged_df["account_name"]

# Add Sub Class column
dim_bs_relation = pd.merge(dim_bs_relation, account_relation, left_on='account_id_lv3', right_on='child', how='left')
dim_bs_relation['account_id_lv2'] = dim_bs_relation['parent']
dim_bs_relation = dim_bs_relation.drop(columns=['parent', 'child'])
merged_df = pd.merge(dim_bs_relation['account_id_lv2'], stock_relation, left_on='account_id_lv2', right_on='account_key', how='left')
dim_bs_relation["Sub Class"] = merged_df["account_name"]

# Add Class column
dim_bs_relation = pd.merge(dim_bs_relation, account_relation, left_on='account_id_lv2', right_on='child', how='left')
dim_bs_relation['account_id_lv1'] = dim_bs_relation['parent']
dim_bs_relation = dim_bs_relation.drop(columns=['parent', 'child'])
merged_df = pd.merge(dim_bs_relation['account_id_lv1'], stock_relation, left_on='account_id_lv1', right_on='account_key', how='left')
dim_bs_relation["Class"] = merged_df["account_name"]

dim_bs_relation

,account_key,account_name,sign,account_type,account_id_lv3,Account,account_id_lv2,Sub Class,account_id_lv1,Class
0,111,Tiền,1,cdkt,110,Tiền và tương đương tiền,100,TÀI SẢN NGẮN HẠN,270,TỔNG TÀI SẢN
1,112,Các khoản tương đương tiền,1,cdkt,110,Tiền và tương đương tiền,100,TÀI SẢN NGẮN HẠN,270,TỔNG TÀI SẢN
2,121,Đầu tư ngắn hạn,1,cdkt,120,Giá trị thuần đầu tư ngắn hạn,100,TÀI SẢN NGẮN HẠN,270,TỔNG TÀI SẢN
3,122,Dự phòng đầu tư ngắn hạn,-1,cdkt,120,Giá trị thuần đầu tư ngắn hạn,100,TÀI SẢN NGẮN HẠN,270,TỔNG TÀI SẢN
4,123,Chứng khoán đầu tư giữ đến ngày đáo hạn,1,cdkt,120,Giá trị thuần đầu tư ngắn hạn,100,TÀI SẢN NGẮN HẠN,270,TỔNG TÀI SẢN
...,...,...,...,...,...,...,...,...,...,...
78,420,Quỹ khác,1,cdkt,410,Vốn và các quỹ,400,VỐN CHỦ SỞ HỮU,440,TỔNG CỘNG NGUỒN VỐN
79,421,Lãi chưa phân phối,1,cdkt,410,Vốn và các quỹ,400,VỐN CHỦ SỞ HỮU,440,TỔNG CỘNG NGUỒN VỐN
80,429,Lợi ích cổ đông không kiểm soát,1,cdkt,410,Vốn và các quỹ,400,VỐN CHỦ SỞ HỮU,440,TỔNG CỘNG NGUỒN VỐN
81,431,Vốn ngân sách nhà nước,1,cdkt,430,Vốn Ngân sách nhà nước và quỹ khác,400,VỐN CHỦ SỞ HỮU,440,TỔNG CỘNG NGUỒN VỐN


In [4]:
dim_is_relation = stock_relation[stock_relation["account_type"] == "kqkd"]

In [13]:
stocks = ['GAS', 'PVD', 'PVT', 'POW', 'PVS', 'PLX', 'PVI']

In [6]:
# Hàm lấy giá cố phiếu
def get_stock_price(stock):
    stock_hist = stock_historical_data(stock, "2020-07-01",
        str(date.today()), "1D", "stock")
    stock_hist["%_change"] = round(((stock_hist['close'] - stock_hist['close'].shift(1))
                                    / stock_hist['close'].shift(1)) * 100, 2)
    stock_hist['change'] = (stock_hist['close'] - stock_hist['close'].shift(1))
    stock_hist['time'] = stock_hist['time'].astype(str)
    stock_hist = stock_hist.rename(columns={'ticker': 'stock_name', 'time': 'date'})

    return stock_hist

In [7]:
get_stock_price('GAS')

,date,open,high,low,close,volume,stock_name,%_change,change
0,2020-07-01,50280,51370,49920,51370,387430,GAS,NaN,NaN
1,2020-07-02,51660,51730,51010,51010,210850,GAS,-0.70,-360.0
2,2020-07-03,51660,51660,50860,50860,310040,GAS,-0.29,-150.0
3,2020-07-06,51950,52670,51520,52380,485000,GAS,2.99,1520.0
4,2020-07-07,52820,53760,52380,52380,819010,GAS,0.00,0.0
...,...,...,...,...,...,...,...,...,...
1021,2024-07-30,77000,77200,76800,77200,631600,GAS,0.26,200.0
1022,2024-07-31,77400,80500,77400,79900,3188600,GAS,3.50,2700.0
1023,2024-08-01,80100,81100,78700,78900,1422900,GAS,-1.25,-1000.0
1024,2024-08-02,78100,78900,77800,78900,1395600,GAS,0.00,0.0


In [19]:
def convert_quarter_to_date(quarter_str):
    quarter, year = quarter_str.split()
    year = int(float(year))
    if quarter == 'Q1':
        return f"{year}-01-01"
    elif quarter == 'Q2':
        return f"{year}-04-01"
    elif quarter == 'Q3':
        return f"{year}-07-01"
    elif quarter == 'Q4':
        return f"{year}-10-01"
    else:
        raise ValueError(f"Invalid quarter: {quarter}")

In [15]:
def transform_report(stock, report_type):
    df = pd.read_excel(f'./Raw Data/{stock}_{report_type}.xlsx', skiprows=7).dropna()

    if report_type == 'IS':
        df = df[:-2]

    df = pd.merge(df, stock_relation, left_on='CHỈ TIÊU', right_on='account_name', how='left')
    df = df.drop(columns=['CHỈ TIÊU', 'account_name', 'sign', 'account_type', 'level'])
    df = df.melt(id_vars=['account_key'], var_name='quarter', value_name='End of Period')
    df['date'] = df['quarter'].apply(convert_quarter_to_date)
    df = df.drop(columns=['quarter'])
    df['stock_name'] = stock

    return df

In [16]:
def get_stock_fundamental_info(stock):
    fundamental_ratio = company_fundamental_ratio(stock) 
    profile = company_profile(stock)
    overview = company_overview(stock)
    evaluation = stock_evaluation(stock)[["ticker", "PE", "industryPE", "PB", "industryPB"]][-1:]

    stock_info = pd.merge(fundamental_ratio[['ticker']], profile[['companyName', 'ticker']], on='ticker', how='inner')
    stock_info = pd.merge(stock_info, overview[['ticker', 'outstandingShare', 'exchange']], on='ticker', how='inner')
    stock_info = pd.merge(stock_info, evaluation, on='ticker', how='inner')

    stock_info = stock_info.rename(columns={'ticker': 'stock_name', 'companyName': 'company_name', 'outstandingShare': 'outstanding_share'})
    stock_info['outstanding_share'] = stock_info['outstanding_share'] * 1000000

    return stock_info

In [17]:
get_stock_fundamental_info('GAS')

,stock_name,company_name,outstanding_share,exchange,PE,industryPE,PB,industryPB
0,GAS,Tổng Công ty Khí Việt Nam - Công ty Cổ phần,2.296700e+09,HOSE,16.6,20.4,2.6,1.8


### Tổng hợp dữ liệu chứng khoán của các công ty

In [22]:
bs_list = [] 
is_list = []
stock_price_list = []
stock_info_list = []

for stock in stocks:
    balance_sheet = transform_report(stock, 'BS')
    income_statement = transform_report(stock, 'IS')
    stock_price = get_stock_price(stock)
    stock_info = get_stock_fundamental_info(stock)
    
    bs_list.append(balance_sheet)
    is_list.append(income_statement)
    stock_price_list.append(stock_price)
    stock_info_list.append(stock_info)

bs_df = pd.concat(bs_list, ignore_index=True)
is_df = pd.concat(is_list, ignore_index=True)
stock_price_df = pd.concat(stock_price_list, ignore_index=True)
stock_info_df = pd.concat(stock_info_list, ignore_index=True)

bs_df

,account_key,End of Period,date,stock_name
0,270,6.308966e+13,2020-10-01,GAS
1,100,3.935442e+13,2020-10-01,GAS
2,110,5.335317e+12,2020-10-01,GAS
3,111,1.323114e+12,2020-10-01,GAS
4,112,4.012203e+12,2020-10-01,GAS
...,...,...,...,...
12220,421A,1.132652e+12,2024-04-01,PVI
12221,421B,6.159965e+11,2024-04-01,PVI
12222,NaN,0.000000e+00,2024-04-01,PVI
12223,NaN,3.518955e+11,2024-04-01,PVI


In [23]:
stock_price_df

,date,open,high,low,close,volume,stock_name,%_change,change
0,2020-07-01,50280,51370,49920,51370,387430,GAS,NaN,NaN
1,2020-07-02,51660,51730,51010,51010,210850,GAS,-0.70,-360.0
2,2020-07-03,51660,51660,50860,50860,310040,GAS,-0.29,-150.0
3,2020-07-06,51950,52670,51520,52380,485000,GAS,2.99,1520.0
4,2020-07-07,52820,53760,52380,52380,819010,GAS,0.00,0.0
...,...,...,...,...,...,...,...,...,...
7177,2024-07-30,53200,53200,52900,53000,184400,PVI,0.00,0.0
7178,2024-07-31,52800,54500,52800,53700,189300,PVI,1.32,700.0
7179,2024-08-01,53200,53200,52500,53000,68300,PVI,-1.30,-700.0
7180,2024-08-02,52000,52400,51300,52000,46400,PVI,-1.89,-1000.0


In [25]:
with pd.ExcelWriter('./Processed data/data.xlsx') as writer:
    dim_bs_relation.to_excel(writer, sheet_name='dim_bs_relation', index=False)
    dim_is_relation.to_excel(writer, sheet_name='dim_is_relation', index=False)
    bs_df.to_excel(writer, sheet_name='cdkt', index=False)
    is_df.to_excel(writer, sheet_name='kqkd', index=False)
    stock_price_df.to_excel(writer, sheet_name='stock_price', index=False)
    stock_info_df.to_excel(writer, sheet_name='stock_info', index=False)